# First Test 

## Each Test is divided by their level . Each level contain their job title test 

## In this test we are going to train feed forward neural network with two hidden layer with each job title data 

### first job title is web development of level 0

First we make data preprocessing templete of level 0 
which will be used by all job title test

their are following steps for data processing
    1. find all vocablory in files
    2. divide all vocablory in two parts
        a. known vocab => which have word2vec in word embeddided file
        b. unknown vocab => which does not have word2vec 
    3. find all known vvocab vector and generate all unknown voacb vector and save in cpickle file
    4. take each sentence from training file and make sentence vector using word2vec file and save it 

In [ ]:
import io
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import os
import numpy as np
import cPickle as pic
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'      # this is used when you want to run on cpu not gpu

lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

In [2]:
def get_vocab(file_name):
    lexicon = []
    with io.open(file_name, 'r', encoding='cp437') as f:
        contents = f.readlines()
        for l in contents:
            all_words = tokenizer.tokenize(l)
            #all_words = word_tokenize(l)
            lexicon += list(all_words)
    lexicon = [(lemmatizer.lemmatize(i)).encode('utf8').lower() for i in lexicon]
    lexicon = sorted(set(lexicon))

    return lexicon

def get_unknown_vocab(fname, words):
    f2 = io.open(fname, 'rb').read()
    vocab = f2.splitlines()
    known_vocab = []
    unknown_vocab = []
    for word in words:
        if word in vocab:
            known_vocab.append(word)
        else:
            unknown_vocab.append(word)
    return known_vocab, unknown_vocab

def get_wiki_glove_vector(fname, words):
    vectors = {}
    W = []
    vocab = {}
    ivocab = {}
    f1 = open(fname, 'rb').read()
    for line in f1.splitlines():
        temp = line.split()
        vectors[temp[0]] = map(float, temp[1:])

    vocab_size = len(words)

    for i in range(len(words)):
        W.append(vectors[words[i]])
        vocab[words[i]] = i
        ivocab[i] = words[i]
    W = np.array(W)
    # normalize each word vector to unit variance
    #print W[0:2], W[-1:-3]
    W_norm = np.zeros(W.shape)
    d = (np.sum(W ** 2, 1) ** (0.5))
    W_norm = (W.T / d).T

    return W_norm, vocab, ivocab


def get_unknown_vec(words, word2vec, vocab, ivocab):
    old_size = len(vocab)
    word2vec = list(word2vec)
    for i in range(len(words)):
        word2vec.append(np.random.uniform(-0.25, 0.25, len(word2vec[0])))
        vocab[words[i]] = i+old_size
        ivocab[i+old_size] = words[i]
    return word2vec, vocab, ivocab

In [3]:
glove_file = '/home/sumit/stanford-nlp/glove/wiki_GVector/glove.6B.100d.txt'
glove_vocab_file = '/home/sumit/stanford-nlp/glove/wiki_GVector/vocab.txt'
word2vec_file = "/home/sumit/PycharmProjects/ResumeParser/Data/resume_segments/Test/web_word2vec.p"
dirlist_path = '/home/sumit/PycharmProjects/ResumeParser/Data/resume_segments/Level0/'
dirlist = os.walk(dirlist_path).next()[2] # only give files not folders
#dirlist = os.listdir(dirlist_path)

if not os.path.isfile(word2vec_file):
    # get vocablory of all file
    vocab = []

    for files in dirlist:
        vocab += get_vocab(dirlist_path+files)
    vocab = sorted(set(vocab))
    print("vocab length: ", len(vocab))
    # get the glove vector
    known_vocab, unknown_vocab = get_unknown_vocab(glove_vocab_file, vocab)
    print("known_vocab length: ", len(known_vocab))
    print("unknown_vocab length: ", len(unknown_vocab))
    Word2vec, vocab, ivocab = get_wiki_glove_vector(glove_file, known_vocab)
    Word2vec, vocab, ivocab = get_unknown_vec(unknown_vocab,Word2vec, vocab, ivocab)
    pic.dump([Word2vec, vocab, ivocab ], open(word2vec_file, 'wb'))
    print("word2vec created")

StopIteration: 

In [ ]:
def get_avg_sen2vec(fname, word2vec_fname, output):
    sen2vec = []
    with open(word2vec_fname, 'rb') as f:
        word2vec, vocab, ivocab = pic.load(f)

    with io.open(fname, 'r', encoding='cp437') as f:
        contents = f.readlines()
        for l in contents:
            temp = np.zeros(len(word2vec[0]), dtype=float)
            all_words = tokenizer.tokenize(l)
            lexicon = list(all_words)
            lexicon = [(lemmatizer.lemmatize(i)).encode('utf8').lower() for i in lexicon]
            for word in lexicon:
                temp += word2vec[vocab[word]]
            sen2vec.append([temp, output])
    return sen2vec

def get_2D_sen2vec(fname, word2vec_fname, output, m):
    sen2vec = []
    with open(word2vec_fname, 'rb') as f:
        word2vec, vocab, ivocab = pic.load(f)

    with io.open(fname, 'r', encoding='cp437') as f:
        contents = f.readlines()
        for l in contents:
            temp = np.zeros([m, len(word2vec[0])], dtype=float)
            count = 0
            all_words = tokenizer.tokenize(l)
            lexicon = list(all_words)
            lexicon = [(lemmatizer.lemmatize(i)).encode('utf8').lower() for i in lexicon]
            for word in lexicon:
                temp[count] = word2vec[vocab[word]]
                count += 1
            sen2vec.append([temp, output])
    return sen2vec

sen2vec_file = "/home/sumit/PycharmProjects/ResumeParser/Data/resume_segments/Test/web_sen2vec.p"


if not os.path.isfile(sen2vec_file):
    # get sentence vector from file
    sen2vec = []
    count  = 0

    for files in dirlist:
        output = [0 for i in range(9)]
        output[count] = 1
        sen2vec +=  get_avg_sen2vec(dirlist_path+files, word2vec_file, output)
        count += 1
    import random
    random.shuffle(sen2vec)
    pic.dump(sen2vec, open(sen2vec_file, 'wb'))
    print("sen2vec created")

In [ ]:
f = io.open(sen2vec_file, 'rb')
sen2vec = pic.load(f)
f.close()
print(len(sen2vec))

train_data = np.array(sen2vec[0:int(len(sen2vec)*0.8)])
test_data = np.array(sen2vec[int(len(sen2vec)*0.8):])
train_x = np.array(list(train_data[:, 0]))
train_y = np.array(list(train_data[:, 1]))
test_x = np.array(list(test_data[:, 0]))
test_y = np.array(list(test_data[:, 1]))
print("train data length: ", len(train_data))
print("test data length: ", len(test_data))
print("structure of train_x", train_x[0])
print("structure of train_y", train_y[0])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 1024, kernel_initializer = 'uniform', activation = 'relu', input_dim = 100))

# Adding the second hidden layer
classifier.add(Dense(units = 512, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(train_x, train_y, batch_size = 30, epochs = 20)

In [ ]:
print dirlist
y_pred = classifier.predict(test_x)


correct = np.equal(np.argmax(y_pred, 1), np.argmax(test_y, 1))
accuracy = np.mean(correct)
print("Accuracy", accuracy)
print "pred: ", np.argmax(y_pred, 1)[0:30]
print "real: ", np.argmax(test_y, 1)[0:30]
print "pred: ", np.argmax(y_pred, 1)[30:65]
print "real: ", np.argmax(test_y, 1)[30:65]
print "pred: ", np.argmax(y_pred, 1)[65:100]
print "real: ", np.argmax(test_y, 1)[65:100]

In [ ]:
sen2vec_file = "/home/sumit/PycharmProjects/ResumeParser/Data/resume_segments/Test/web_sen2vec2D.p"


if not os.path.isfile(sen2vec_file):
    # get sentence vector from file
    sen2vec = []
    count  = 0
    for files in dirlist:
        output = [0 for i in range(9)]
        output[count] = 1
        sen2vec +=  get_2D_sen2vec(dirlist_path+files, word2vec_file, output, 50)
        count += 1
    import random
    random.shuffle(sen2vec)
    pic.dump(sen2vec, open(sen2vec_file, 'wb'))
    print("sen2vec created")

In [ ]:
f = io.open(sen2vec_file, 'rb')
sen2vec = pic.load(f)
f.close()
print(len(sen2vec))

train_data = np.array(sen2vec[0:int(len(sen2vec)*0.8)])
test_data = np.array(sen2vec[int(len(sen2vec)*0.8):])
train_x = np.array(list(train_data[:, 0]))
train_y = np.array(list(train_data[:, 1]))
test_x = np.array(list(test_data[:, 0]))
test_y = np.array(list(test_data[:, 1]))
print("train data length: ", len(train_data))
print("test data length: ", len(test_data))
print("structure of train_x", train_x[0])
print("structure of train_y", train_y[0])
train_x = np.reshape(train_x, [-1, 50, 100, 1])
test_x = np.reshape(test_x, [-1, 50, 100, 1])
print(train_x.shape, test_x.shape)

In [ ]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
cnn_classifier = Sequential()

# Step 1 - Convolution
cnn_classifier.add(Conv2D(50, (4, 100), input_shape = (50, 100, 1), activation = 'relu'))

# Step 2 - Pooling
cnn_classifier.add(AveragePooling2D(pool_size = (2, 1)))

# Step 3 - Flattening
cnn_classifier.add(Flatten())

# Step 4 - Full connection
cnn_classifier.add(Dense(units = 1024, activation = 'relu'))
cnn_classifier.add(Dense(units = 512, activation = 'relu'))
cnn_classifier.add(Dense(units = 9, activation = 'softmax'))

# Compiling the CNN
cnn_classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
cnn_classifier.fit(train_x, train_y, batch_size = 30, epochs = 20)

In [ ]:
print dirlist
y_pred = cnn_classifier.predict(test_x)


correct = np.equal(np.argmax(y_pred, 1), np.argmax(test_y, 1))
accuracy = np.mean(correct)
print("Accuracy", accuracy)
print "pred: ", np.argmax(y_pred, 1)[0:30]
print "real: ", np.argmax(test_y, 1)[0:30]
print "pred: ", np.argmax(y_pred, 1)[30:65]
print "real: ", np.argmax(test_y, 1)[30:65]
print "pred: ", np.argmax(y_pred, 1)[65:100]
print "real: ", np.argmax(test_y, 1)[65:100]

Now we take any resume and predict the output 

In [ ]:
resume_file = "/home/sumit/PycharmProjects/ResumeParser/Data/resume_segments/Test/sample.txt"
# get vocablory of all file
vocab = get_vocab(resume_file)
print("vocab length: ", len(vocab))
# get the glove vector
known_vocab, unknown_vocab = get_unknown_vocab(glove_vocab_file, vocab)
print("known_vocab length: ", len(known_vocab))
print("unknown_vocab length: ", len(unknown_vocab))
Word2vec, vocab, ivocab = get_wiki_glove_vector(glove_file, known_vocab)
Word2vec, vocab, ivocab = get_unknown_vec(unknown_vocab,Word2vec, vocab, ivocab)

In [ ]:
def get_avg_Testsen2vec(fname, word2vec, vocab, ivocab):
    sen2vec = []

    with io.open(fname, 'r', encoding='cp437') as f:
        contents = f.readlines()
        for l in contents:
            temp = np.zeros(len(word2vec[0]), dtype=float)
            all_words = tokenizer.tokenize(l)
            lexicon = list(all_words)
            lexicon = [(lemmatizer.lemmatize(i)).encode('utf8').lower() for i in lexicon]
            for word in lexicon:
                temp += word2vec[vocab[word]]
            sen2vec.append(temp)
    return sen2vec

def get_2D_Testsen2vec(fname,  word2vec, vocab, ivocab , m):
    sen2vec = []

    with io.open(fname, 'r', encoding='cp437') as f:
        contents = f.readlines()
        for l in contents:
            temp = np.zeros([m, len(word2vec[0])], dtype=float)
            count = 0
            all_words = tokenizer.tokenize(l)
            lexicon = list(all_words)
            lexicon = [(lemmatizer.lemmatize(i)).encode('utf8').lower() for i in lexicon]
            for word in lexicon:
                temp[count] = word2vec[vocab[word]]
                count += 1
            sen2vec.append(temp)
    return sen2vec

In [ ]:
sen2vec = get_avg_Testsen2vec(resume_file, Word2vec, vocab, ivocab)
print dirlist

y_pred = classifier.predict(np.array(sen2vec))
with open(resume_file, 'r') as f:
    content = f.readlines()
count = 0
for line in content:
    print line ,'=>', dirlist[np.argmax(y_pred, 1)[count]][:-4]
    count += 1

accuracy = 85%

In [ ]:
sen2vec = get_2D_Testsen2vec(resume_file, Word2vec, vocab, ivocab, 50)
print dirlist
sen2vec = np.reshape(np.array(sen2vec), (len(sen2vec), 50, 100, 1))
y_pred = cnn_classifier.predict(np.array(sen2vec))
with open(resume_file, 'r') as f:
    content = f.readlines()
count = 0
for line in content:
    print line ,'=>', dirlist[np.argmax(y_pred, 1)[count]][:-4]
    count += 1

In [ ]:
sen2vec = get_avg_Testsen2vec(resume_file, Word2vec, vocab, ivocab)
pred = classifier.predict(np.array(sen2vec))
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
pred = sc_x.fit_transform(pred)
index = np.array([[i] for i in range(1, len(sen2vec)+1)])
pred = np.append(pred, index, axis=1)

print pred[0:5]
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
wcss = []
for i in range(1, 11):
    cluster = KMeans(n_clusters=i)
    cluster.fit(pred)
    wcss.append(cluster.inertia_)

plt.plot(range(1, 11), wcss)
plt.show()
for j in range(9):
    cluster = KMeans(n_clusters=3)
    y_pred = cluster.fit_predict(pred[:, [j, 9]])
    print pred[0, [j, 9]]
    color = ['red', 'blue', 'green', 'pink', 'black']

    for i in range(5):
        plt.scatter(pred[y_pred==i, j], pred[y_pred==i, 9], color=color[i], s=100)
    #plt.scatter(cluster.cluster_centers_[:,0], cluster.cluster_centers_[:,1], s=300, color='yellow')
    plt.show()
    print y_pred

In [ ]:
sen2vec = get_avg_Testsen2vec(resume_file, Word2vec, vocab, ivocab)
pred = classifier.predict(np.array(sen2vec))
#from sklearn.preprocessing import StandardScaler
#sc_x = StandardScaler()
#pred = sc_x.fit_transform(pred)
index = np.array([[i] for i in range(1, len(sen2vec)+1)])
pred = np.append(pred, index, axis=1)

print pred[0:5]
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as  sch
denfrograms = sch.dendrogram(sch.linkage(pred, method='ward'))
plt.show()

plt.plot(range(1, 11), wcss)
plt.show()
for j in range(9):
    from sklearn.cluster import AgglomerativeClustering as AC
    cluster = AC(n_clusters=3)
    y_pred = cluster.fit_predict(pred[:, [j, 9]])
    print pred[0, [j, 9]]
    color = ['red', 'blue', 'green', 'pink', 'black']

    for i in range(5):
        plt.scatter(pred[y_pred==i, j], pred[y_pred==i, 9], color=color[i], s=100)
    #plt.scatter(cluster.cluster_centers_[:,0], cluster.cluster_centers_[:,1], s=300, color='yellow')
    plt.show()
    print y_pred
print "full"
cluster = AC(n_clusters=3)
y_pred = cluster.fit_predict(pred)
print y_pred

accuracy = 77.5%